# OSNet（外観特徴量）

### 出力形式
- label
- point_ID
- 外観特徴量512次元


またdata_pathは以下のディレクトリ構成とする．各point_IDディレクトリの中に画像ファイルを格納している．

- exp1
    ... (5 directories omitted)
    - exp1_annotation
        - crops
            - person
                ... (4 directories omitted)
                - ichimanda
                    ... (4 directories omitted)
                    - 1_249
                    - 2_1
                - ichiraku
                    ... (7 directories omitted)
                    - 1_57
                    - 1_63
    - exp1_point1
        - crops
            - person
                - exp1_point1_other
                    ... (122 directories omitted)
                    - 100
                    - 102
- exp2
    ... (5 directories omitted)
    - exp2_annotation
        - crops
            - person
                ... (4 directories omitted)
                - ichimanda
                    ... (3 directories omitted)
                    - 1_1
                    - 1_15
                - ichiraku
                    ... (4 directories omitted)
                    - 1_161
                    - 1_162
    - exp2_point1
        - crops
            - person
                - exp2_point1_other
                    ... (142 directories omitted)
                    - 111
                    - 113

In [ ]:
# osnet関連のライブラリ
import torchreid
from torchreid.reid.utils import FeatureExtractor

import cv2
import numpy as np
import glob
from natsort import natsorted # https://suda-memo.hatenablog.com/entry/2021/07/23/192209

import os
import csv
import pandas as pd

# track内のフォルダのリストを取得
# もとはtrack_dir - for exp in ~~で間はなかった
# track_path ="C:/Users/sugie/PycharmProjects/facenet_and_osnet/data/runs/track/"
# track_list = os.listdir(track_path)
# track_dir = [f for f in track_list if os.path.isdir(os.path.join(track_path, f))]

# 複数ファイル（augとか）をやるとき用
data_path = "C:/Users/sugie/PycharmProjects/StrongSORT-YOLO/aug_data/aug_blur_data"
data_list = os.listdir(data_path)
data_dir = [f for f in data_list if os.path.isdir(os.path.join(data_path, f))]
for track_name in data_dir:
    track_path = data_path + "/" + track_name
    track_list = os.listdir(track_path)
    track_dir = [f for f in track_list if os.path.isdir(os.path.join(track_path, f))]
    # person内のフォルダリスト取得
    for exp in track_dir:
      person_path= track_path + "/" + exp + "/crops/person/"
      filelist = []
      for d in os.listdir(person_path):
       if os.path.isdir(os.path.join(person_path, d)):
        filelist.append(d)
      person_list = os.listdir(person_path) # person内のフォルダリスト
      person_dir = [f for f in natsorted(person_list) if os.path.isdir(os.path.join(person_path, f))]

      # ID内の画像の全パスを取得
      for name in person_dir:
          name_path = person_path + name
          name_list = os.listdir(name_path)  # name内のリスト
          name_dir = [f for f in natsorted(name_list) if os.path.isdir(os.path.join(name_path, f))]
          firstloop = True
          for ID in name_dir:
              ID_path = glob.glob(name_path + "/" + ID + "/**")  # ID_path=画像の絶対パスのリスト
              # if	str.isalpha(ID): # 不要画像フォルダtempがあるとき用
              #     pass
              # else:
              extractor = FeatureExtractor(
                  model_name='osnet_x1_0',
                  # model_name='mobilenetv2_x1_0',
                  model_path='a/b/c/model.pth.tar',
                  device='cuda',  # device="cpu"にすればgpuなしでも行ける。もとはcuda
                  # image_size=(256, 128),
                  image_size=(128, 64),
              )

              features = extractor(ID_path)
              app_features = features.to('cpu').detach().numpy().copy()

              # 保存先の用意
              save_csv_dir = "./" + track_name + "_app_csv/" + "csv_" + name
              os.makedirs(save_csv_dir, exist_ok=True)
              save_csv_name = track_name + "_app_features_" + name + ".csv"

              # ndarrayをdataframeに変換して行列名をつけてcsv保存
              file_name_col = [os.path.basename(file_name) for file_name in ID_path]
              point_ID = [os.path.basename(os.path.dirname(file_name)) for file_name in ID_path]
              if firstloop:
                  dimension = [i for i in range(512)]
                  df = pd.DataFrame(app_features, index=[file_name_col])
                  df.insert(loc=0, column='file_name', value=file_name_col)
                  df.insert(loc=0, column='point_ID', value=point_ID)
                  firstloop = False
              else:
                  df2 = pd.DataFrame(app_features)
                  df2.insert(loc=0, column='file_name', value=file_name_col)
                  df2.insert(loc=0, column='point_ID', value=point_ID)
                  df = pd.concat([df, df2])
          df.to_csv(save_csv_dir + "/" + save_csv_name, index=False)
    #名前は適宜変更する
    # csvの結合
    #save_csv_name = os.path.basename(track_path) + "_app_all.csv"
    save_csv_name = os.path.basename(track_path) + "_app_all.csv"
    # csv_path = "C:/Users/sugie/PycharmProjects/facenet_and_osnet/deep-person-reid/" + track_name + "_face_csv" "/" + track_name + "_all"
    # os.makedirs(csv_path, exist_ok=True)
    #csv_path = "C:/Users/sugie/PycharmProjects/my-StrongSORT-YOLO/deep-person-reid/" + track_name + "_app_csv"
    csv_path = "C:/Users/sugie/PycharmProjects/StrongSORT-YOLO/" + track_name + "_app_csv"
    #exp()は場合によって変えてください
    csv_list = os.listdir(csv_path)
    firstloop = True
    count = 0
    for csv in csv_list:
        csv_namelist = glob.glob(csv_path + "/" + csv + "/**")
        if not csv_namelist:
            print(f"No files found in {csv_path}/{csv}")
            continue
        csv_name = csv_namelist[0]
        count_col_list = [0]
        if firstloop:
            df = pd.read_csv(csv_name)  # つなげたいファイルを書く。日本語を含むcsvを開くときはencoding="cp932"をつける
            count_col = count_col_list * len(df)
            df.insert(loc=0, column='label', value=count_col)
            firstloop = False
        elif "other" in csv_name:
            df2 = pd.read_csv(csv_name)
            count_col2 = count_col_list * len(df2)
            df2.insert(loc=0, column="label", value=count_col2)
            df = pd.concat([df, df2])
        else:
            count = count + 1
            count_col_list2 = [count]
            df2 = pd.read_csv(csv_name)
            count_col2 = count_col_list2 * len(df2)
            df2.insert(loc=0, column="label", value=count_col2)
            df = pd.concat([df, df2])
    df.to_csv(csv_path + "/" + save_csv_name, index=False)

